# Demo of kernel methods library

In this notebook we present the various important components of the `kernelmethods` library, and provide some example usage scenarios.

This library consists of a set of key classes such as `KernelMatrix`, a diverse library of kernel functions, as well as meta classes like `KernelSet` and `KernelBucket` to manage an array of kernel matrices. In addition, a library of kernel operations and related utilities are included.


## Table of Contents
- [Kernel functions](#kerfuncs)
- [Kernel matrix](#kernelmatrix)
- [Attributes for kernel matrix](#attr_km)


## Kernel functions <a name="kerfuncs"></a> 

Let's get started with some **kernel functions**!

A [kernel function](https://en.wikipedia.org/wiki/Positive-definite_kernel) takes in two samples (each represented by an array of values in their raw input space) as inputs and computes their inner product. In a typical machine learning library, these kernel functions are usually directly implemented as mathematical formulas, that blindly compute and return the inner product. They are almost always without any structure, validation or representation associated with them, which can be a recipe for invalid or disastrous implementations. In this library, given kernel functions are key to and at the core of everything, we take a more concerted effort to enforce certain structure, uniform validation and readable representation. We achieve this by defining a `BaseKernelFunction` abstract base class and making each kernel function inherit from it.

The `BaseKernelFunction` enforces each derived kernel:
1. to be callable, with two inputs
2. to have a name and a str representation
3. provides a method to check whether the derived kernel func is a valid kernel i.e. the kernel matrix derived on a random sample is positive semi-definite (PSD)
4. and that it is symmetric (via tests) as required.

These properties can be verified using the built-in kernel functions such as `PolyKernel` and `GaussianKernel` e.g. 

In [1]:
from kernelmethods import PolyKernel, GaussianKernel

poly = PolyKernel(degree=4)
rbf = GaussianKernel()
# you can print/present then in many ways
print(poly)
print(rbf)
repr(rbf)

polynomial(degree=4,b=0)
gaussian(sigma=2.0)


'gaussian(sigma=2.0)'

Each of these are "children" of `BaseKernelFunction`, and hence have the aforementioned desirable properties and relevant attributes such as *degree* and intercept (*b*), a name and validation on input data:

In [2]:
print([poly.degree, poly.b, poly.name])
# which can also be conveniently presented via its repr or str form
print(poly)

[4, 0, 'polynomial']
polynomial(degree=4,b=0)


They can be called with two vectorial inputs, which returns their input product:

In [3]:
import numpy as np
x = np.array([1, 2, 3])
y = np.array([2, 3, 4])
poly(x, y)

160000

There is an internal input validation - which throws a `ValueError` if the data is not of the right type or valid

In [4]:
poly([1, 2, 3], [4, 5, 'a'])

ValueError: input data type <U21 is not compatible with the required <class 'numpy.number'>

More importantly, every kernel function has a method to verify that kernel function is valid (KM induced is PSD!):

In [5]:
poly.is_psd()

True

In addition to using the pre-defined classes, one can easily build new classes either by defining new classes themselves (starting from `BaseKernelFunction` or its derived classes), or by simply specifying a callable and using the `KernelFromCallable` convenience class. For example, you a new type of polynomial kernel without an intercept, that can be achieved via

In [6]:
from kernelmethods.base import BaseKernelFunction, KernelFromCallable

# define that function
def poly_no_intercept(x, y, degree=2):
    return x.dot(y.T) ** degree

new_poly = KernelFromCallable(input_func=poly_no_intercept)
print(new_poly)

poly_no_intercept


You can check that `new_poly` is indeed a KernelFunction:

In [7]:
isinstance(new_poly, BaseKernelFunction)

True

Now, we can check its properties and usability:

In [8]:
new_poly(x, y)

400

You can also quickly check if this new function is a valid [mercer kenel](https://en.wikipedia.org/wiki/Mercer%27s_theorem) or not:

In [9]:
new_poly.is_psd()

True

In [10]:
# you will see the rbf is also pSD
rbf.is_psd()

True

## Kernel matrix <a name="kernelmatrix"></a>

The gram matrix resulting from the pairwise application of the kernel function results in what is called the kernel matrix. This is a key data structure for all the kernel methods and learning algorithms. We designed `KernelMatrix` to make it self-contained, efficient and yet generic. 

You can import it simply by:

In [11]:
from kernelmethods import KernelMatrix

An instance can be created by specifying which function to be used as the kernel, and an optional name:

In [12]:
km = KernelMatrix(rbf)

You can inspect its properties easily, and get an easy to read representation anytime:

In [13]:
km

KernelMatrix: gaussian(sigma=2.0)

Specifying a kernel function is not enough - we usually need to attach and apply it to a sample. Let's create a simple dataset consisting of 10 points with 4 features each:

In [14]:
sample = np.random.rand(10, 4)

np.set_printoptions(precision=2)
print(sample)

[[0.89 0.31 0.59 0.8 ]
 [0.08 0.64 0.55 0.59]
 [0.5  0.29 0.88 0.01]
 [0.92 0.95 0.9  0.1 ]
 [0.03 0.75 0.52 0.92]
 [0.88 0.41 0.9  0.77]
 [0.57 0.72 0.37 0.42]
 [0.36 0.19 0.05 0.13]
 [0.68 0.42 0.4  0.28]
 [0.18 0.9  0.21 0.69]]


Attaching it is as easy as 

In [15]:
km.attach_to(sample)

You can then see it is ready for use, with a clear repr:

In [16]:
km

KernelMatrix: gaussian(sigma=2.0) (normed=True) on sample (10, 4)

You can display the full matrix simply with the `.full` attribute:

In [17]:
km.full

array([[1.  , 0.9 , 0.9 , 0.88, 0.89, 0.99, 0.94, 0.88, 0.96, 0.88],
       [0.9 , 1.  , 0.91, 0.86, 0.98, 0.9 , 0.96, 0.91, 0.94, 0.98],
       [0.9 , 0.91, 1.  , 0.93, 0.84, 0.91, 0.92, 0.91, 0.96, 0.84],
       [0.88, 0.86, 0.93, 1.  , 0.81, 0.91, 0.93, 0.82, 0.93, 0.84],
       [0.89, 0.98, 0.84, 0.81, 1.  , 0.88, 0.93, 0.85, 0.89, 0.98],
       [0.99, 0.9 , 0.91, 0.91, 0.88, 1.  , 0.93, 0.84, 0.94, 0.86],
       [0.94, 0.96, 0.92, 0.93, 0.93, 0.93, 1.  , 0.94, 0.98, 0.97],
       [0.88, 0.91, 0.91, 0.82, 0.85, 0.84, 0.94, 1.  , 0.96, 0.9 ],
       [0.96, 0.94, 0.96, 0.93, 0.89, 0.94, 0.98, 0.96, 1.  , 0.92],
       [0.88, 0.98, 0.84, 0.84, 0.98, 0.86, 0.97, 0.9 , 0.92, 1.  ]])

Notice that the kernel matrices are `normalized` by default, and hence all the diagonal elements are `1.0`. If you have a specific requirement not to normalize it, you can choose `normalized=False` during instantiation.

which then allows retrieval of various properties including size, full kernel matrix or portions of it in a convenient manner.

In [24]:
print('number of elements: {}, \n\tsamples: {} '.format(km.size, km.num_samples))
print('shape of KM: ', km.shape)

number of elements: 100, 
	samples: 10 
shape of KM:  (10, 10)


as well as easily evaluate its frobenius norm:

In [18]:
km.frob_norm

9.208181961556187

In addition, this class offers the following public attributes of KM (including methods):

In [19]:
[ a for a in dir(km)  if not a.startswith('_') ]

['attach_to',
 'attributes',
 'center',
 'centered',
 'diagonal',
 'frob_norm',
 'full',
 'full_sparse',
 'get_attr',
 'kernel',
 'name',
 'normalize',
 'normed_km',
 'num_samples',
 'set_attr',
 'shape',
 'size']

You can also perform common operations such as centering, and normalization quite easily, via `km.center()` and  `km.normalize()`, as well as query its diagonal with `km.diagonal()`

## Attributes for KernelMatrix <a name="attr_km"></a>

Another cool feature of the `KernelMatrix` class is the ability to attach arbitary user-defined attributes, for easy identification of a given kernel matrix. This is especially handy when one has to traverse among a large collection of kernel matrices, and programmatic identification is necessary!

For example, you can identify the KM with the source of the dataset:

In [29]:
km.set_attr('source', 'random')

or anything else you wish, like properties:

In [31]:
km.set_attr('properties', ['sigma', 4])

You can easily retrieve all the properties with `.attributes()` which returns the internal dictionary:

In [32]:
km.attributes()

{'source': 'random', 'properties': ['sigma', 4]}

Or just the one you like via `.get_attr()`:

In [33]:
km.get_attr('source')

'random'

The utility of these will be more obvious when dealing with [`KernelBucket` and `KernelSet`](#kmcollections)